In [1]:
import lpips
import torch
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

Measuring values and saving to csv file

In [2]:
# Initialize LPIPS model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = lpips.LPIPS(net='alex').to(device)

# Transform: convert image to tensor and normalize to [-1, 1]
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Optional: adjust to match your data
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # LPIPS expects input in [-1, 1]
])

root_dir = 'Distortions'
results = []

for scene_folder in os.listdir(root_dir):
    scene_path = os.path.join(root_dir, scene_folder)
    if not os.path.isdir(scene_path):
        continue

    # Find the reference image (ends with '0.bmp')
    reference_image = None
    for file in os.listdir(scene_path):
        if file.endswith('_0.bmp'):
            reference_image = os.path.join(scene_path, file)
            break

    if reference_image is None:
        print(f"No reference image found in {scene_path}")
        continue

    ref_img = transform(Image.open(reference_image).convert('RGB')).unsqueeze(0).to(device)

    # Loop over distortion type folders (Blur, Noise, etc.)
    for distortion_folder in os.listdir(scene_path):
        distortion_path = os.path.join(scene_path, distortion_folder)
        if not os.path.isdir(distortion_path):
            continue

        for file in os.listdir(distortion_path):
            if not file.endswith('.bmp'):
                continue

            distorted_image_path = os.path.join(distortion_path, file)

            # Load and preprocess distorted image
            dist_img = transform(Image.open(distorted_image_path).convert('RGB')).unsqueeze(0).to(device)

            # Compute LPIPS distance
            with torch.no_grad():
                distance = loss_fn(ref_img, dist_img).item()

            # Save result
            results.append({
                'Scene': scene_folder,
                'DistortionType': distortion_folder,
                'Filename': file,
                'LPIPS': distance
            })

# Save all results to CSV
df = pd.DataFrame(results)
df.to_csv('lpips_distortion_results.csv', index=False)
print("✅ Results saved to lpips_distortion_results.csv")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\Nocto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Nocto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\Nocto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lpips\weights\v0.1\alex.pth
✅ Results saved to lpips_distortion_results.csv


Creating charts

In [3]:
# === Load CSV ===
df = pd.read_csv("lpips_distortion_results.csv")

# Extract numeric distortion level from filename
def extract_level(filename):
    match = re.search(r'_(\d+)\.bmp$', filename)
    return int(match.group(1)) if match else None

df['Level'] = df['Filename'].apply(extract_level)
df = df.dropna(subset=['Level'])
df['Level'] = df['Level'].astype(int)

# === Parse Markdown for level-to-value mapping ===
def parse_md_description(md_path):
    with open(md_path, 'r') as f:
        lines = f.readlines()

    mapping = {}
    current_type = None

    for line in lines:
        line = line.strip()

        if line.startswith("##"):
            header = line[2:].strip()
            # Remove number and any parentheses
            match = re.match(r'\d+\.\s*([^(]+)', header)
            if match:
                current_type = match.group(1).strip()
                mapping[current_type] = {}
        elif line.startswith("- Level"):
            match = re.match(r"- Level (\d+): (.+)", line)
            if match and current_type:
                level = int(match.group(1))
                desc = match.group(2).strip()
                mapping[current_type][level] = desc
    return mapping

desc_mapping = parse_md_description("Image distorion Description.md")

# Add synthetic LPIPS=0 for level 0 reference images
reference_rows = []
for scene in df['Scene'].unique():
    for distortion in df['DistortionType'].unique():
        reference_rows.append({
            'Scene': scene,
            'DistortionType': distortion,
            'Filename': 'image_0.bmp',
            'LPIPS': 0.0,
            'Level': 0
        })

df = pd.concat([df, pd.DataFrame(reference_rows)], ignore_index=True)

# === Plotting ===
output_dir = "charts"
os.makedirs(output_dir, exist_ok=True)

for distortion in df['DistortionType'].unique():
    plt.figure(figsize=(8, 5))
    distortion_df = df[df['DistortionType'] == distortion]

    # Get value mapping
    value_map = desc_mapping.get(distortion, {})
    x_vals = [0] + [value_map.get(i, str(i)) for i in range(1, 5)]

    # For consistent X-axis order
    x_vals_str = [str(v) for v in x_vals]

    for scene in sorted(distortion_df['Scene'].unique()):
        scene_df = distortion_df[distortion_df['Scene'] == scene]
        scene_df = scene_df.groupby('Level')['LPIPS'].mean().sort_index()

        # Build y and x values
        y = [scene_df.get(lvl, None) for lvl in range(0, 5)]
        plt.plot(x_vals_str, y, marker='o', label=scene)

    plt.title(f'LPIPS vs Distortion Value – {distortion}')
    plt.xlabel('Distortion Parameter')
    plt.ylabel('LPIPS Distance')
    plt.legend(title='Scene', fontsize='small')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/{distortion}_lpips_multiscene_chart.png')
    plt.close()

print("✅ All charts saved in 'charts/' with correct labels and scene-wise lines.")

✅ All charts saved in 'charts/' with correct labels and scene-wise lines.


Creating group chart

In [ ]:
# === Load CSV and filter for _4.bmp ===
df = pd.read_csv("lpips_distortion_results.csv")
df = df[df['Filename'].str.endswith('_4.bmp')]
df['Level'] = 4

# === Parse level 4 descriptions from markdown ===
def parse_md_level_4_descriptions(md_path):
    with open(md_path, 'r') as f:
        lines = f.readlines()

    descriptions = {}
    current_type = None
    for line in lines:
        line = line.strip()
        if line.startswith("##"):
            header = line[2:].strip()
            match = re.match(r'\d+\.\s*([^\(]+)', header)
            if match:
                current_type = match.group(1).strip()
        elif line.startswith("- Level 4:") and current_type:
            desc = line.split(":", 1)[1].strip()
            descriptions[current_type] = desc
    return descriptions

desc_mapping = parse_md_level_4_descriptions("Image distorion Description.md")

# === Clean distortion names and attach descriptions ===
df['DistortionTypeClean'] = df['DistortionType'].str.strip()
df['Description'] = df['DistortionTypeClean'].map(desc_mapping)
df['Label'] = df.apply(
    lambda row: f"{row['DistortionTypeClean']} ({row['Description']})" if pd.notnull(row['Description']) else row['DistortionTypeClean'],
    axis=1
)

# === Get unique distortions and scenes ===
distortions = sorted(df['Label'].unique())
scenes = sorted(df['Scene'].unique())

# === Prepare grouped bar chart data ===
bar_width = 0.8 / len(scenes)  # fit all scene bars per group
x = np.arange(len(distortions))  # X positions for each group

colors = plt.cm.tab10.colors
fig, ax = plt.subplots(figsize=(14, 6))

# Plot bars per scene
for i, scene in enumerate(scenes):
    heights = []
    for dist in distortions:
        match = df[(df['Label'] == dist) & (df['Scene'] == scene)]
        heights.append(match['LPIPS'].values[0] if not match.empty else 0)

    ax.bar(x + i * bar_width, heights, bar_width, label=scene, color=colors[i % len(colors)])

# === Final plot formatting ===
ax.set_xticks(x + bar_width * (len(scenes) - 1) / 2)
ax.set_xticklabels(distortions, rotation=45, ha='right')
ax.set_ylabel("LPIPS Distance")
ax.set_title("LPIPS Scores at Distortion Level 4 (Grouped by Distortion Type)")
ax.legend(title="Scene")
plt.tight_layout()
plt.savefig("charts/bar_chart_level4_grouped_by_distortion.png")
plt.show()
